# Get Data

In [4]:
from keras.datasets import reuters

(x_train, y_train), (x_test, y_test) = reuters.load_data(path='reuters.npz',
                                                         num_words=None,
                                                         skip_top=0,
                                                         maxlen=None,
                                                         test_split=0.2,
                                                         seed=113,
                                                         start_char=1,
                                                         oov_char=2,
                                                         index_from=3)

# Explore Data

In [5]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)
print(x_train[0][0:10])
print(y_train)

(8982,)
(8982,)
(2246,)
(2246,)
[1, 27595, 28842, 8, 43, 10, 447, 5, 25, 207]
[ 3  4  3 ... 25  3 25]


# Process Data

Turns lists of integers into a 2D integer tensor of shape (samples, maxlen):

In [6]:
from keras.preprocessing.sequence import pad_sequences

maxlen = 100

x_train = pad_sequences(x_train, maxlen=maxlen)
x_test = pad_sequences(x_test, maxlen=maxlen)

In [7]:
print(x_train.shape)
print(x_test.shape)

(8982, 100)
(2246, 100)


Turn labels to one-hot encodings:

In [8]:
from keras.utils.np_utils import to_categorical

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [9]:
print(y_train.shape)
print(y_test.shape)

(8982, 46)
(2246, 46)


# Initial Models

## Fully-connected network

In [15]:
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense

max_features = max(x_train.shape[0], x_test.shape[0])
num_categories = y_train.shape[1]

model = Sequential()
model.add(Embedding(max_features, 64, input_length=maxlen))
model.add(Flatten())
model.add(Dense(32, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(num_categories, activation='softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 100, 64)           574848    
_________________________________________________________________
flatten_2 (Flatten)          (None, 6400)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 32)                204832    
_________________________________________________________________
dense_5 (Dense)              (None, 64)                2112      
_________________________________________________________________
dense_6 (Dense)              (None, 46)                2990      
Total params: 784,782
Trainable params: 784,782
Non-trainable params: 0
_________________________________________________________________


In [16]:
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['acc'])

history = model.fit(x_train, y_train,
                    epochs=15,
                    batch_size=32,
                    validation_split=0.2)

Train on 7185 samples, validate on 1797 samples
Epoch 1/15
7185/7185 [==============================] - 2s 220us/step - loss: 1.9520 - acc: 0.4843 - val_loss: 1.6292 - val_acc: 0.5893
Epoch 2/15
7185/7185 [==============================] - 1s 164us/step - loss: 1.3692 - acc: 0.6564 - val_loss: 1.4164 - val_acc: 0.6550
Epoch 3/15
7185/7185 [==============================] - 1s 168us/step - loss: 0.9102 - acc: 0.7783 - val_loss: 1.3522 - val_acc: 0.6878
Epoch 4/15
7185/7185 [==============================] - 1s 162us/step - loss: 0.5889 - acc: 0.8632 - val_loss: 1.4478 - val_acc: 0.6856
Epoch 5/15
7185/7185 [==============================] - 1s 162us/step - loss: 0.3809 - acc: 0.9168 - val_loss: 1.5439 - val_acc: 0.6750
Epoch 6/15
7185/7185 [==============================] - 1s 161us/step - loss: 0.2543 - acc: 0.9460 - val_loss: 1.6756 - val_acc: 0.6700
Epoch 7/15
7185/7185 [==============================] - 1s 159us/step - loss: 0.1964 - acc: 0.9571 - val_loss: 1.7036 - val_acc: 0.6683


Notes:
- Not much increase in validation accuracy after the 3-4 epoch, and the validation loss stops improving around then as well.
- Starts overfitting around the 2nd epoch

## RNN (Simple)

In [17]:
from keras.layers import SimpleRNN

model = Sequential()
model.add(Embedding(max_features, 64, input_length=maxlen))
model.add(SimpleRNN(64))
model.add(Dense(num_categories, activation='softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 100, 64)           574848    
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, 64)                8256      
_________________________________________________________________
dense_7 (Dense)              (None, 46)                2990      
Total params: 586,094
Trainable params: 586,094
Non-trainable params: 0
_________________________________________________________________


In [18]:
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['acc'])

history = model.fit(x_train, y_train,
                    epochs=10,
                    batch_size=32,
                    validation_split=0.2)

Train on 7185 samples, validate on 1797 samples
Epoch 1/10
7185/7185 [==============================] - 24s 3ms/step - loss: 2.4271 - acc: 0.3851 - val_loss: 2.0715 - val_acc: 0.4758
Epoch 2/10
7185/7185 [==============================] - 23s 3ms/step - loss: 1.9442 - acc: 0.4932 - val_loss: 1.9474 - val_acc: 0.4769
Epoch 3/10
7185/7185 [==============================] - 23s 3ms/step - loss: 1.6323 - acc: 0.5807 - val_loss: 1.8935 - val_acc: 0.4925
Epoch 4/10
7185/7185 [==============================] - 23s 3ms/step - loss: 1.3968 - acc: 0.6482 - val_loss: 1.8075 - val_acc: 0.5292
Epoch 5/10
7185/7185 [==============================] - 25s 3ms/step - loss: 1.1746 - acc: 0.7088 - val_loss: 1.8524 - val_acc: 0.5353
Epoch 6/10
7185/7185 [==============================] - 22s 3ms/step - loss: 0.9624 - acc: 0.7651 - val_loss: 1.8745 - val_acc: 0.5448
Epoch 7/10
7185/7185 [==============================] - 23s 3ms/step - loss: 0.7960 - acc: 0.8088 - val_loss: 1.9355 - val_acc: 0.5487
Epoch 8

Notes:
- Computationally expensive and performed worse than the simple Dense network
- Validation loss stopped improving at around the 7th epoch
- Starts overfitting around the 3rd epoch

## RNN (LSTM)

In [19]:
from keras.layers import LSTM

model = Sequential()
model.add(Embedding(max_features, 64, input_length=maxlen))
model.add(LSTM(64))
model.add(Dense(num_categories, activation='softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 100, 64)           574848    
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                33024     
_________________________________________________________________
dense_8 (Dense)              (None, 46)                2990      
Total params: 610,862
Trainable params: 610,862
Non-trainable params: 0
_________________________________________________________________


In [20]:
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['acc'])

history = model.fit(x_train, y_train,
                    epochs=10,
                    batch_size=32,
                    validation_split=0.2)

Train on 7185 samples, validate on 1797 samples
Epoch 1/10
7185/7185 [==============================] - 55s 8ms/step - loss: 2.1743 - acc: 0.4288 - val_loss: 1.9566 - val_acc: 0.4775
Epoch 2/10
7185/7185 [==============================] - 53s 7ms/step - loss: 1.7618 - acc: 0.5272 - val_loss: 1.7151 - val_acc: 0.5281
Epoch 3/10
7185/7185 [==============================] - 53s 7ms/step - loss: 1.6227 - acc: 0.5585 - val_loss: 1.6056 - val_acc: 0.5765
Epoch 4/10
7185/7185 [==============================] - 52s 7ms/step - loss: 1.5152 - acc: 0.5925 - val_loss: 1.5660 - val_acc: 0.5821
Epoch 5/10
7185/7185 [==============================] - 53s 7ms/step - loss: 1.3921 - acc: 0.6223 - val_loss: 1.5234 - val_acc: 0.5960
Epoch 6/10
7185/7185 [==============================] - 52s 7ms/step - loss: 1.2873 - acc: 0.6468 - val_loss: 1.4520 - val_acc: 0.6283
Epoch 7/10
7185/7185 [==============================] - 54s 7ms/step - loss: 1.1873 - acc: 0.6782 - val_loss: 1.4599 - val_acc: 0.6377
Epoch 8

In [21]:
model.save_weights('rnn_lstm_model.h5')

Notes:
- Computationally expensive and performed about the same as the Dense network
- Validation loss was decreasing and accuracy was continuing to rise, could probably increase by a few more points by just running for more epochs
- Started to overfit a bit around epoch 5

## RNN (GRU)

In [22]:
from keras.layers import GRU

model = Sequential()
model.add(Embedding(max_features, 64, input_length=maxlen))
model.add(GRU(64))
model.add(Dense(num_categories, activation='softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 100, 64)           574848    
_________________________________________________________________
gru_1 (GRU)                  (None, 64)                24768     
_________________________________________________________________
dense_9 (Dense)              (None, 46)                2990      
Total params: 602,606
Trainable params: 602,606
Non-trainable params: 0
_________________________________________________________________


In [23]:
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['acc'])

history = model.fit(x_train, y_train,
                    epochs=10,
                    batch_size=32,
                    validation_split=0.2)

Train on 7185 samples, validate on 1797 samples
Epoch 1/10
7185/7185 [==============================] - 49s 7ms/step - loss: 2.2044 - acc: 0.3914 - val_loss: 1.8770 - val_acc: 0.4864
Epoch 2/10
7185/7185 [==============================] - 47s 7ms/step - loss: 1.7607 - acc: 0.5299 - val_loss: 1.7969 - val_acc: 0.5314
Epoch 3/10
7185/7185 [==============================] - 47s 7ms/step - loss: 1.6245 - acc: 0.5720 - val_loss: 1.6462 - val_acc: 0.5615
Epoch 4/10
7185/7185 [==============================] - 47s 7ms/step - loss: 1.5351 - acc: 0.5946 - val_loss: 1.5845 - val_acc: 0.5821
Epoch 5/10
7185/7185 [==============================] - 47s 6ms/step - loss: 1.4626 - acc: 0.6143 - val_loss: 1.5717 - val_acc: 0.5838
Epoch 6/10
7185/7185 [==============================] - 48s 7ms/step - loss: 1.3946 - acc: 0.6358 - val_loss: 1.5791 - val_acc: 0.6027
Epoch 7/10
7185/7185 [==============================] - 47s 7ms/step - loss: 1.3174 - acc: 0.6596 - val_loss: 1.5125 - val_acc: 0.6277
Epoch 8

In [24]:
model.save_weights('rnn_gru_model.h5')

Notes:
- Computationally expensive and slightly worse than the Dense network
- Validation loss was decreasing and accuracy was continuing to rise, could probably increase by a few more points by just running for more epochs
- Started to overfit a bit around epoch 5

## One-Dimensional CNN

In [27]:
from keras.layers import Conv1D, MaxPooling1D, GlobalMaxPooling1D

model = Sequential()
model.add(Embedding(max_features, 64, input_length=maxlen))
model.add(Conv1D(64, 7, activation='relu'))
model.add(MaxPooling1D(5))
model.add(Conv1D(64, 7, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(num_categories, activation='softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 100, 64)           574848    
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 94, 64)            28736     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 18, 64)            0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 12, 64)            28736     
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 64)                0         
_________________________________________________________________
dense_11 (Dense)             (None, 46)                2990      
Total params: 635,310
Trainable params: 635,310
Non-trainable params: 0
_________________________________________________________________


In [28]:
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['acc'])

history = model.fit(x_train, y_train,
                    epochs=10,
                    batch_size=32,
                    validation_split=0.2)

Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Train on 7185 samples, validate on 1797 samples
Epoch 1/10
7185/7185 [==============================] - 4s 494us/step - loss: 1.9897 - acc: 0.4852 - val_loss: 1.6320 - val_acc: 0.5815
Epoch 2/10
7185/7185 [==============================] - 1s 176us/step - loss: 1.5421 - acc: 0.6058 - val_loss: 1.5099 - val_acc: 0.6394
Epoch 3/10
7185/7185 [==============================] - 1s 175us/step - loss: 1.3303 - acc: 0.6763 - val_loss: 1.3942 - val_acc: 0.6811
Epoch 4/10
7185/7185 [==============================] - 1s 179us/step - loss: 1.1304 - acc: 0.7265 - val_loss: 1.3550 - val_acc: 0.6912
Epoch 5/10
7185/7185 [==============================] - 1s 177us/step - loss: 0.9719 - acc: 0.7635 - val_loss: 1.2935 - val_acc: 0.7028
Epoch 6/10
7185/7185 [==============================] - 1s 180us/step - loss: 0.8302 - acc: 0.8011 - val_loss: 1.3607 - val_acc: 0.7162
Epoch 7/10
7185/7185 [==============================] - 1s

Notes:
- Achieved better results than all other models with less than one second per epoch
- Validation loss minimized at around epoch 5
- Started overfitting around epoch 4

## Combination Network

Combining the Conv1D layer with the best-performing RNN so far (LSTM):

In [31]:
model = Sequential()
model.add(Embedding(max_features, 64, input_length=maxlen))
model.add(Conv1D(64, 7, activation='relu'))
model.add(MaxPooling1D(5))
model.add(Conv1D(64, 7, activation='relu'))
model.add(LSTM(64))
model.add(Dense(num_categories, activation='softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_10 (Embedding)     (None, 100, 64)           574848    
_________________________________________________________________
conv1d_6 (Conv1D)            (None, 94, 64)            28736     
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 18, 64)            0         
_________________________________________________________________
conv1d_7 (Conv1D)            (None, 12, 64)            28736     
_________________________________________________________________
lstm_3 (LSTM)                (None, 64)                33024     
_________________________________________________________________
dense_12 (Dense)             (None, 46)                2990      
Total params: 668,334
Trainable params: 668,334
Non-trainable params: 0
_________________________________________________________________


In [32]:
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['acc'])

history = model.fit(x_train, y_train,
                    epochs=10,
                    batch_size=32,
                    validation_split=0.2)

Train on 7185 samples, validate on 1797 samples
Epoch 1/10
7185/7185 [==============================] - 10s 1ms/step - loss: 1.9330 - acc: 0.4907 - val_loss: 1.7330 - val_acc: 0.5454
Epoch 2/10
7185/7185 [==============================] - 8s 1ms/step - loss: 1.8547 - acc: 0.5157 - val_loss: 1.8262 - val_acc: 0.5303
Epoch 3/10
7185/7185 [==============================] - 8s 1ms/step - loss: 1.7761 - acc: 0.5495 - val_loss: 1.7590 - val_acc: 0.5370
Epoch 4/10
7185/7185 [==============================] - 8s 1ms/step - loss: 1.6350 - acc: 0.5930 - val_loss: 1.6729 - val_acc: 0.5771
Epoch 5/10
7185/7185 [==============================] - 8s 1ms/step - loss: 1.5303 - acc: 0.6166 - val_loss: 1.6462 - val_acc: 0.5715
Epoch 6/10
7185/7185 [==============================] - 8s 1ms/step - loss: 1.5084 - acc: 0.6195 - val_loss: 1.6109 - val_acc: 0.5954
Epoch 7/10
7185/7185 [==============================] - 8s 1ms/step - loss: 1.8803 - acc: 0.5230 - val_loss: 1.8761 - val_acc: 0.5314
Epoch 8/10
71

Notes:
- Stops improving after around epoch 6
- Only slight overfitting starting around epoch 8
- Not very computationally-expensive

# Improved Models

## Fully-connected network v2

Adding a dropout layer, removing one Dense layer, increasing nodes on first Dense layer:

In [52]:
from keras.layers import Dropout

model = Sequential()
model.add(Embedding(max_features, 64, input_length=maxlen))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_categories, activation='softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_20 (Embedding)     (None, 100, 64)           574848    
_________________________________________________________________
flatten_12 (Flatten)         (None, 6400)              0         
_________________________________________________________________
dense_37 (Dense)             (None, 64)                409664    
_________________________________________________________________
dropout_10 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_38 (Dense)             (None, 46)                2990      
Total params: 987,502
Trainable params: 987,502
Non-trainable params: 0
_________________________________________________________________


In [53]:
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['acc'])

history = model.fit(x_train, y_train,
                    epochs=10,
                    batch_size=32,
                    validation_split=0.2)

Train on 7185 samples, validate on 1797 samples
Epoch 1/10
7185/7185 [==============================] - 3s 408us/step - loss: 2.0944 - acc: 0.4810 - val_loss: 1.6085 - val_acc: 0.6027
Epoch 2/10
7185/7185 [==============================] - 1s 182us/step - loss: 1.4451 - acc: 0.6434 - val_loss: 1.4232 - val_acc: 0.6594
Epoch 3/10
7185/7185 [==============================] - 1s 182us/step - loss: 1.0965 - acc: 0.7257 - val_loss: 1.3318 - val_acc: 0.6912
Epoch 4/10
7185/7185 [==============================] - 1s 181us/step - loss: 0.8307 - acc: 0.7921 - val_loss: 1.3543 - val_acc: 0.6923
Epoch 5/10
7185/7185 [==============================] - 1s 183us/step - loss: 0.6137 - acc: 0.8458 - val_loss: 1.4435 - val_acc: 0.6956
Epoch 6/10
7185/7185 [==============================] - 1s 185us/step - loss: 0.4714 - acc: 0.8806 - val_loss: 1.4987 - val_acc: 0.7051
Epoch 7/10
7185/7185 [==============================] - 1s 188us/step - loss: 0.3755 - acc: 0.9031 - val_loss: 1.6171 - val_acc: 0.6978


Notes:
- 0.7 appears to be about our max validation accuracy using only the simple Dense network

## RNN (Simple) v2

Adding recurrent dropout:

In [58]:
model = Sequential()
model.add(Embedding(max_features, 64, input_length=maxlen))
model.add(SimpleRNN(32,
                    dropout=0.2,
                    recurrent_dropout=0.2,
                    return_sequences=True))
model.add(SimpleRNN(64,
                    dropout=0.2,
                    recurrent_dropout=0.2))
model.add(Dense(num_categories, activation='softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_23 (Embedding)     (None, 100, 64)           574848    
_________________________________________________________________
simple_rnn_5 (SimpleRNN)     (None, 100, 32)           3104      
_________________________________________________________________
simple_rnn_6 (SimpleRNN)     (None, 64)                6208      
_________________________________________________________________
dense_41 (Dense)             (None, 46)                2990      
Total params: 587,150
Trainable params: 587,150
Non-trainable params: 0
_________________________________________________________________


In [59]:
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['acc'])

history = model.fit(x_train, y_train,
                    epochs=10,
                    batch_size=32,
                    validation_split=0.2)

Train on 7185 samples, validate on 1797 samples
Epoch 1/10
7185/7185 [==============================] - 50s 7ms/step - loss: 2.6511 - acc: 0.3108 - val_loss: 2.4038 - val_acc: 0.3450
Epoch 2/10
7185/7185 [==============================] - 50s 7ms/step - loss: 2.3871 - acc: 0.3628 - val_loss: 2.2888 - val_acc: 0.3806
Epoch 3/10
7185/7185 [==============================] - 49s 7ms/step - loss: 2.2890 - acc: 0.4014 - val_loss: 2.2766 - val_acc: 0.4124
Epoch 4/10
7185/7185 [==============================] - 45s 6ms/step - loss: 2.2228 - acc: 0.4273 - val_loss: 2.1460 - val_acc: 0.4346
Epoch 5/10
7185/7185 [==============================] - 45s 6ms/step - loss: 2.1589 - acc: 0.4523 - val_loss: 2.0936 - val_acc: 0.4585
Epoch 6/10
7185/7185 [==============================] - 45s 6ms/step - loss: 2.0701 - acc: 0.4721 - val_loss: 2.0650 - val_acc: 0.4619
Epoch 7/10
7185/7185 [==============================] - 45s 6ms/step - loss: 2.0031 - acc: 0.4885 - val_loss: 2.0515 - val_acc: 0.4374
Epoch 8

In [60]:
model.save_weights('rnn_simple_model.h5')

Notes:
- Not showing much promise with this model.  Training is time-consuming and it is rarely getting even close to the Dense model's accuracy.

## RNN (LSTM) v2

Add dropout:

In [61]:
model = Sequential()
model.add(Embedding(max_features, 64, input_length=maxlen))
model.add(LSTM(64,
               dropout=0.2,
               recurrent_dropout=0.2))
model.add(Dense(num_categories, activation='softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_24 (Embedding)     (None, 100, 64)           574848    
_________________________________________________________________
lstm_4 (LSTM)                (None, 64)                33024     
_________________________________________________________________
dense_42 (Dense)             (None, 46)                2990      
Total params: 610,862
Trainable params: 610,862
Non-trainable params: 0
_________________________________________________________________


In [62]:
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['acc'])

history = model.fit(x_train, y_train,
                    epochs=10,
                    batch_size=32,
                    validation_split=0.2)

Train on 7185 samples, validate on 1797 samples
Epoch 1/10
7185/7185 [==============================] - 58s 8ms/step - loss: 2.2902 - acc: 0.4122 - val_loss: 1.9923 - val_acc: 0.4841
Epoch 2/10
7185/7185 [==============================] - 56s 8ms/step - loss: 1.8659 - acc: 0.5048 - val_loss: 1.7412 - val_acc: 0.5498
Epoch 3/10
7185/7185 [==============================] - 55s 8ms/step - loss: 1.7251 - acc: 0.5499 - val_loss: 1.6971 - val_acc: 0.5526
Epoch 4/10
7185/7185 [==============================] - 56s 8ms/step - loss: 1.6349 - acc: 0.5717 - val_loss: 1.6333 - val_acc: 0.5787
Epoch 5/10
7185/7185 [==============================] - 55s 8ms/step - loss: 1.5302 - acc: 0.5997 - val_loss: 1.5627 - val_acc: 0.5999
Epoch 6/10
7185/7185 [==============================] - 56s 8ms/step - loss: 1.4365 - acc: 0.6242 - val_loss: 1.5147 - val_acc: 0.6149
Epoch 7/10
7185/7185 [==============================] - 55s 8ms/step - loss: 1.3513 - acc: 0.6452 - val_loss: 1.4534 - val_acc: 0.6322
Epoch 8

In [63]:
model.save_weights('rnn_lstm_model.h5')

Notes:
- Good progress so far on this model, going to continue training

In [64]:
model.load_weights('rnn_lstm_model.h5')

model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['acc'])

history = model.fit(x_train, y_train,
                    epochs=10,
                    batch_size=32,
                    validation_split=0.2)

Train on 7185 samples, validate on 1797 samples
Epoch 1/10
7185/7185 [==============================] - 59s 8ms/step - loss: 1.0562 - acc: 0.7253 - val_loss: 1.3535 - val_acc: 0.6750
Epoch 2/10
7185/7185 [==============================] - 56s 8ms/step - loss: 0.9881 - acc: 0.7446 - val_loss: 1.3286 - val_acc: 0.6867
Epoch 3/10
7185/7185 [==============================] - 55s 8ms/step - loss: 0.9369 - acc: 0.7569 - val_loss: 1.3484 - val_acc: 0.6900
Epoch 4/10
7185/7185 [==============================] - 56s 8ms/step - loss: 0.8896 - acc: 0.7705 - val_loss: 1.3316 - val_acc: 0.6978
Epoch 5/10
7185/7185 [==============================] - 56s 8ms/step - loss: 0.8434 - acc: 0.7825 - val_loss: 1.3290 - val_acc: 0.7040
Epoch 6/10
7185/7185 [==============================] - 56s 8ms/step - loss: 0.7992 - acc: 0.7925 - val_loss: 1.3337 - val_acc: 0.7073
Epoch 7/10
7185/7185 [==============================] - 56s 8ms/step - loss: 0.7647 - acc: 0.8022 - val_loss: 1.3331 - val_acc: 0.7001
Epoch 8

In [65]:
model.save_weights('rnn_lstm_model.h5')

Notes:
- Starts to overfit significantly during these epochs, although the validation accuracy still got to over 0.7, which makes this the best model so far.
- Going to try training one more LSTM model with more agressive dropout rates and stacked LSTM layers

In [66]:
model = Sequential()
model.add(Embedding(max_features, 64, input_length=maxlen))
model.add(LSTM(64,
               dropout=0.1,
               recurrent_dropout=0.5,
               return_sequences=True))
model.add(LSTM(64,
               dropout=0.1,
               recurrent_dropout=0.5))
model.add(Dense(num_categories, activation='softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_25 (Embedding)     (None, 100, 64)           574848    
_________________________________________________________________
lstm_5 (LSTM)                (None, 100, 64)           33024     
_________________________________________________________________
lstm_6 (LSTM)                (None, 64)                33024     
_________________________________________________________________
dense_43 (Dense)             (None, 46)                2990      
Total params: 643,886
Trainable params: 643,886
Non-trainable params: 0
_________________________________________________________________


In [67]:
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['acc'])

history = model.fit(x_train, y_train,
                    epochs=10,
                    batch_size=32,
                    validation_split=0.2)

Train on 7185 samples, validate on 1797 samples
Epoch 1/10
7185/7185 [==============================] - 114s 16ms/step - loss: 2.1939 - acc: 0.4195 - val_loss: 1.8000 - val_acc: 0.5203
Epoch 2/10
7185/7185 [==============================] - 111s 15ms/step - loss: 1.7913 - acc: 0.5225 - val_loss: 1.7737 - val_acc: 0.5331
Epoch 3/10
7185/7185 [==============================] - 111s 15ms/step - loss: 1.7304 - acc: 0.5420 - val_loss: 1.7106 - val_acc: 0.5676
Epoch 4/10
7185/7185 [==============================] - 111s 15ms/step - loss: 1.6460 - acc: 0.5680 - val_loss: 1.6867 - val_acc: 0.5765
Epoch 5/10
7185/7185 [==============================] - 111s 15ms/step - loss: 1.5863 - acc: 0.5866 - val_loss: 1.6527 - val_acc: 0.5793
Epoch 6/10
7185/7185 [==============================] - 112s 16ms/step - loss: 1.5107 - acc: 0.6032 - val_loss: 1.5994 - val_acc: 0.5965
Epoch 7/10
7185/7185 [==============================] - 111s 15ms/step - loss: 1.4453 - acc: 0.6269 - val_loss: 1.5726 - val_acc: 

In [68]:
model.save_weights('rnn_lstm_model_v2.h5')

Notes:
- Good progress so far on this model, going to continue training

In [69]:
model.load_weights('rnn_lstm_model_v2.h5')

model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['acc'])

history = model.fit(x_train, y_train,
                    epochs=10,
                    batch_size=32,
                    validation_split=0.2)

Train on 7185 samples, validate on 1797 samples
Epoch 1/10
7185/7185 [==============================] - 115s 16ms/step - loss: 1.2490 - acc: 0.6800 - val_loss: 1.4700 - val_acc: 0.6333
Epoch 2/10
7185/7185 [==============================] - 110s 15ms/step - loss: 1.1918 - acc: 0.6910 - val_loss: 1.4658 - val_acc: 0.6433
Epoch 3/10
7185/7185 [==============================] - 111s 15ms/step - loss: 1.1427 - acc: 0.7038 - val_loss: 1.4426 - val_acc: 0.6533
Epoch 4/10
7185/7185 [==============================] - 112s 16ms/step - loss: 1.0881 - acc: 0.7228 - val_loss: 1.4356 - val_acc: 0.6572
Epoch 5/10
7185/7185 [==============================] - 111s 15ms/step - loss: 1.0505 - acc: 0.7278 - val_loss: 1.4535 - val_acc: 0.6555
Epoch 6/10
7185/7185 [==============================] - 111s 15ms/step - loss: 1.0081 - acc: 0.7392 - val_loss: 1.4638 - val_acc: 0.6555
Epoch 7/10
7185/7185 [==============================] - 113s 16ms/step - loss: 0.9781 - acc: 0.7431 - val_loss: 1.4218 - val_acc: 

In [70]:
model.save_weights('rnn_lstm_model_v2.h5')

Notes:
- Starts to overfit significantly during these epochs.  Model stops improving after around
- May want to try one more model with even higher regularization values (run for 20 epochs)

In [78]:
model = Sequential()
model.add(Embedding(max_features, 64, input_length=maxlen))
model.add(LSTM(64,
               dropout=0.5,
               recurrent_dropout=0.5,
               return_sequences=True))
model.add(LSTM(64,
               dropout=0.5,
               recurrent_dropout=0.5))
model.add(Dense(num_categories, activation='softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_30 (Embedding)     (None, 100, 64)           574848    
_________________________________________________________________
lstm_17 (LSTM)               (None, 100, 64)           33024     
_________________________________________________________________
lstm_18 (LSTM)               (None, 64)                33024     
_________________________________________________________________
dense_48 (Dense)             (None, 46)                2990      
Total params: 643,886
Trainable params: 643,886
Non-trainable params: 0
_________________________________________________________________


In [79]:
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['acc'])

history = model.fit(x_train, y_train,
                    epochs=10,
                    batch_size=32,
                    validation_split=0.2)

Train on 7185 samples, validate on 1797 samples
Epoch 1/10
7185/7185 [==============================] - 117s 16ms/step - loss: 2.2638 - acc: 0.4036 - val_loss: 1.8574 - val_acc: 0.5114
Epoch 2/10
7185/7185 [==============================] - 111s 15ms/step - loss: 1.8338 - acc: 0.5113 - val_loss: 1.9191 - val_acc: 0.5136
Epoch 3/10
7185/7185 [==============================] - 112s 16ms/step - loss: 1.7580 - acc: 0.5321 - val_loss: 1.7615 - val_acc: 0.5437
Epoch 4/10
7185/7185 [==============================] - 111s 15ms/step - loss: 1.7092 - acc: 0.5482 - val_loss: 1.7613 - val_acc: 0.5537
Epoch 5/10
7185/7185 [==============================] - 111s 15ms/step - loss: 1.6662 - acc: 0.5621 - val_loss: 1.6917 - val_acc: 0.5648
Epoch 6/10
7185/7185 [==============================] - 111s 15ms/step - loss: 1.6212 - acc: 0.5777 - val_loss: 1.6614 - val_acc: 0.5849
Epoch 7/10
7185/7185 [==============================] - 111s 16ms/step - loss: 1.5825 - acc: 0.5898 - val_loss: 1.6249 - val_acc: 

In [80]:
model.save_weights('rnn_lstm_model_v3.h5')

In [81]:
model.load_weights('rnn_lstm_model_v3.h5')

model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['acc'])

history = model.fit(x_train, y_train,
                    epochs=10,
                    batch_size=32,
                    validation_split=0.2)

Train on 7185 samples, validate on 1797 samples
Epoch 1/10
7185/7185 [==============================] - 116s 16ms/step - loss: 1.4361 - acc: 0.6387 - val_loss: 1.5252 - val_acc: 0.6244
Epoch 2/10
7185/7185 [==============================] - 111s 15ms/step - loss: 1.3948 - acc: 0.6494 - val_loss: 1.5181 - val_acc: 0.6333
Epoch 3/10
7185/7185 [==============================] - 111s 15ms/step - loss: 1.3559 - acc: 0.6543 - val_loss: 1.4908 - val_acc: 0.6405
Epoch 4/10
7185/7185 [==============================] - 111s 15ms/step - loss: 1.3243 - acc: 0.6619 - val_loss: 1.4692 - val_acc: 0.6416
Epoch 5/10
7185/7185 [==============================] - 111s 15ms/step - loss: 1.2847 - acc: 0.6701 - val_loss: 1.4744 - val_acc: 0.6439
Epoch 6/10
7185/7185 [==============================] - 111s 15ms/step - loss: 1.2603 - acc: 0.6763 - val_loss: 1.4456 - val_acc: 0.6394
Epoch 7/10
7185/7185 [==============================] - 111s 15ms/step - loss: 1.2289 - acc: 0.6866 - val_loss: 1.4758 - val_acc: 

In [82]:
model.save_weights('rnn_lstm_model_v3.h5')

Notes:
- Still improving.  Running for an additional 10 epochs

In [83]:
model.load_weights('rnn_lstm_model_v3.h5')

model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['acc'])

history = model.fit(x_train, y_train,
                    epochs=10,
                    batch_size=32,
                    validation_split=0.2)

Train on 7185 samples, validate on 1797 samples
Epoch 1/10
7185/7185 [==============================] - 119s 17ms/step - loss: 1.1301 - acc: 0.7056 - val_loss: 1.4057 - val_acc: 0.6605
Epoch 2/10
7185/7185 [==============================] - 125s 17ms/step - loss: 1.1083 - acc: 0.7086 - val_loss: 1.4176 - val_acc: 0.6550
Epoch 3/10
7185/7185 [==============================] - 112s 16ms/step - loss: 1.0811 - acc: 0.7177 - val_loss: 1.4214 - val_acc: 0.6617
Epoch 4/10
7185/7185 [==============================] - 113s 16ms/step - loss: 1.0604 - acc: 0.7219 - val_loss: 1.4449 - val_acc: 0.6667
Epoch 5/10
7185/7185 [==============================] - 116s 16ms/step - loss: 1.0366 - acc: 0.7299 - val_loss: 1.4277 - val_acc: 0.6711
Epoch 6/10
7185/7185 [==============================] - 114s 16ms/step - loss: 1.0240 - acc: 0.7303 - val_loss: 1.4549 - val_acc: 0.6661
Epoch 7/10
7185/7185 [==============================] - 112s 16ms/step - loss: 1.0090 - acc: 0.7367 - val_loss: 1.4374 - val_acc: 

In [84]:
model.save_weights('rnn_lstm_model_v3.h5')

Notes:
- The validation accuracy is improving very slightly still, but the model is starting to severely overfit and the computational time is getting too large.  Going to use the first version of the improved model moving forward.

## RNN (GRU) v2

Add dropout:

In [85]:
model = Sequential()
model.add(Embedding(max_features, 64, input_length=maxlen))
model.add(GRU(64,
              dropout=0.2,
              recurrent_dropout=0.2))
model.add(Dense(num_categories, activation='softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_31 (Embedding)     (None, 100, 64)           574848    
_________________________________________________________________
gru_2 (GRU)                  (None, 64)                24768     
_________________________________________________________________
dense_49 (Dense)             (None, 46)                2990      
Total params: 602,606
Trainable params: 602,606
Non-trainable params: 0
_________________________________________________________________


In [86]:
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['acc'])

history = model.fit(x_train, y_train,
                    epochs=10,
                    batch_size=32,
                    validation_split=0.2)

Train on 7185 samples, validate on 1797 samples
Epoch 1/10
7185/7185 [==============================] - 51s 7ms/step - loss: 2.2493 - acc: 0.3942 - val_loss: 1.9054 - val_acc: 0.4808
Epoch 2/10
7185/7185 [==============================] - 51s 7ms/step - loss: 1.8086 - acc: 0.5168 - val_loss: 1.7545 - val_acc: 0.5376
Epoch 3/10
7185/7185 [==============================] - 48s 7ms/step - loss: 1.7019 - acc: 0.5493 - val_loss: 1.7222 - val_acc: 0.5587
Epoch 4/10
7185/7185 [==============================] - 49s 7ms/step - loss: 1.6391 - acc: 0.5716 - val_loss: 1.6703 - val_acc: 0.5676
Epoch 5/10
7185/7185 [==============================] - 50s 7ms/step - loss: 1.5918 - acc: 0.5858 - val_loss: 1.6373 - val_acc: 0.5938
Epoch 6/10
7185/7185 [==============================] - 51s 7ms/step - loss: 1.5323 - acc: 0.6008 - val_loss: 1.5991 - val_acc: 0.5838
Epoch 7/10
7185/7185 [==============================] - 51s 7ms/step - loss: 1.4761 - acc: 0.6141 - val_loss: 1.5609 - val_acc: 0.6004
Epoch 8

In [87]:
model.save_weights('rnn_gru_model.h5')

Notes:
- Training well so far.  Going to keep it going...

In [88]:
model.load_weights('rnn_gru_model.h5')

model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['acc'])

history = model.fit(x_train, y_train,
                    epochs=10,
                    batch_size=32,
                    validation_split=0.2)

Train on 7185 samples, validate on 1797 samples
Epoch 1/10
7185/7185 [==============================] - 55s 8ms/step - loss: 1.2436 - acc: 0.6711 - val_loss: 1.4316 - val_acc: 0.6366
Epoch 2/10
7185/7185 [==============================] - 52s 7ms/step - loss: 1.1971 - acc: 0.6888 - val_loss: 1.3769 - val_acc: 0.6550
Epoch 3/10
7185/7185 [==============================] - 48s 7ms/step - loss: 1.1388 - acc: 0.7074 - val_loss: 1.3827 - val_acc: 0.6539
Epoch 4/10
7185/7185 [==============================] - 49s 7ms/step - loss: 1.0921 - acc: 0.7151 - val_loss: 1.3349 - val_acc: 0.6756
Epoch 5/10
7185/7185 [==============================] - 49s 7ms/step - loss: 1.0402 - acc: 0.7342 - val_loss: 1.3152 - val_acc: 0.6800
Epoch 6/10
7185/7185 [==============================] - 51s 7ms/step - loss: 0.9971 - acc: 0.7467 - val_loss: 1.2968 - val_acc: 0.6873
Epoch 7/10
7185/7185 [==============================] - 51s 7ms/step - loss: 0.9594 - acc: 0.7564 - val_loss: 1.2734 - val_acc: 0.6945
Epoch 8

In [89]:
model.save_weights('rnn_gru_model.h5')

Notes:
- Model look ok, but not as effective as the LSTM.  Will use that one moving forward

## One-Dimensional CNN v2

In [143]:
model = Sequential()
model.add(Embedding(max_features, 64, input_length=maxlen))
model.add(Conv1D(64, 7, activation='relu'))
model.add(MaxPooling1D(5))
model.add(Conv1D(64, 7, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dropout(0.5))
model.add(Dense(num_categories, activation='softmax'))

model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['acc'])

history = model.fit(x_train, y_train,
                    epochs=15,
                    batch_size=32,
                    validation_split=0.2)

Train on 7185 samples, validate on 1797 samples
Epoch 1/15
7185/7185 [==============================] - 11s 2ms/step - loss: 2.1115 - acc: 0.4593 - val_loss: 1.6841 - val_acc: 0.5615
Epoch 2/15
7185/7185 [==============================] - 2s 216us/step - loss: 1.6694 - acc: 0.5726 - val_loss: 1.5914 - val_acc: 0.5915
Epoch 3/15
7185/7185 [==============================] - 2s 217us/step - loss: 1.5088 - acc: 0.6132 - val_loss: 1.5676 - val_acc: 0.6199
Epoch 4/15
7185/7185 [==============================] - 2s 216us/step - loss: 1.3651 - acc: 0.6618 - val_loss: 1.4396 - val_acc: 0.6711
Epoch 5/15
7185/7185 [==============================] - 2s 220us/step - loss: 1.2181 - acc: 0.7037 - val_loss: 1.4245 - val_acc: 0.6767
Epoch 6/15
7185/7185 [==============================] - 2s 220us/step - loss: 1.1023 - acc: 0.7255 - val_loss: 1.4184 - val_acc: 0.6895
Epoch 7/15
7185/7185 [==============================] - 2s 217us/step - loss: 1.0053 - acc: 0.7421 - val_loss: 1.4070 - val_acc: 0.6967
E

In [144]:
model.save_weights('1d_cnn_model.h5')

Notes:
- Still incredibly fast training times, although the accuracy is not much better than the non-dropout version.

## Combination Network v2

In [105]:
model = Sequential()
model.add(Embedding(max_features, 64, input_length=maxlen))
model.add(Conv1D(64, 7, activation='relu'))
model.add(MaxPooling1D(5))
model.add(Conv1D(64, 7, activation='relu'))
model.add(LSTM(64,
               dropout=0.2,
               recurrent_dropout=0.2))
model.add(Dense(num_categories, activation='softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_40 (Embedding)     (None, 100, 64)           574848    
_________________________________________________________________
conv1d_27 (Conv1D)           (None, 94, 64)            28736     
_________________________________________________________________
max_pooling1d_14 (MaxPooling (None, 18, 64)            0         
_________________________________________________________________
conv1d_28 (Conv1D)           (None, 12, 64)            28736     
_________________________________________________________________
lstm_19 (LSTM)               (None, 64)                33024     
_________________________________________________________________
dense_55 (Dense)             (None, 46)                2990      
Total params: 668,334
Trainable params: 668,334
Non-trainable params: 0
_________________________________________________________________


In [106]:
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['acc'])

history = model.fit(x_train, y_train,
                    epochs=10,
                    batch_size=32,
                    validation_split=0.2)

Train on 7185 samples, validate on 1797 samples
Epoch 1/10
7185/7185 [==============================] - 15s 2ms/step - loss: 1.9717 - acc: 0.4814 - val_loss: 1.7859 - val_acc: 0.5314
Epoch 2/10
7185/7185 [==============================] - 9s 1ms/step - loss: 1.6075 - acc: 0.5685 - val_loss: 1.5449 - val_acc: 0.6016
Epoch 3/10
7185/7185 [==============================] - 10s 1ms/step - loss: 1.5988 - acc: 0.5829 - val_loss: 2.0398 - val_acc: 0.4919
Epoch 4/10
7185/7185 [==============================] - 9s 1ms/step - loss: 1.7183 - acc: 0.5566 - val_loss: 1.6259 - val_acc: 0.5832
Epoch 5/10
7185/7185 [==============================] - 9s 1ms/step - loss: 1.4844 - acc: 0.6159 - val_loss: 1.5672 - val_acc: 0.5965
Epoch 6/10
7185/7185 [==============================] - 9s 1ms/step - loss: 1.3858 - acc: 0.6418 - val_loss: 1.5210 - val_acc: 0.6138
Epoch 7/10
7185/7185 [==============================] - 9s 1ms/step - loss: 1.3090 - acc: 0.6633 - val_loss: 1.5141 - val_acc: 0.6283
Epoch 8/10
7

In [107]:
model.save_weights('combo_model.h5')

Notes:
- Model is starting to overfit, but will continue training for a bit to see where it goes

In [108]:
model.load_weights('combo_model.h5')

model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['acc'])

history = model.fit(x_train, y_train,
                    epochs=10,
                    batch_size=32,
                    validation_split=0.2)

Train on 7185 samples, validate on 1797 samples
Epoch 1/10
7185/7185 [==============================] - 15s 2ms/step - loss: 1.0638 - acc: 0.7340 - val_loss: 1.4204 - val_acc: 0.6583
Epoch 2/10
7185/7185 [==============================] - 9s 1ms/step - loss: 1.0055 - acc: 0.7523 - val_loss: 1.4332 - val_acc: 0.6572
Epoch 3/10
7185/7185 [==============================] - 9s 1ms/step - loss: 0.9603 - acc: 0.7573 - val_loss: 1.4320 - val_acc: 0.6550
Epoch 4/10
7185/7185 [==============================] - 10s 1ms/step - loss: 0.9184 - acc: 0.7713 - val_loss: 1.4462 - val_acc: 0.6594
Epoch 5/10
7185/7185 [==============================] - 10s 1ms/step - loss: 1.0632 - acc: 0.7346 - val_loss: 1.5013 - val_acc: 0.6516
Epoch 6/10
7185/7185 [==============================] - 10s 1ms/step - loss: 0.9812 - acc: 0.7541 - val_loss: 1.5020 - val_acc: 0.6589
Epoch 7/10
7185/7185 [==============================] - 10s 1ms/step - loss: 0.9299 - acc: 0.7658 - val_loss: 1.5612 - val_acc: 0.6466
Epoch 8/1

In [109]:
model.save_weights('combo_model.h5')

Notes:
- Severe overfitting.  Going to play around with different regularization options

In [137]:
model = Sequential()
model.add(Embedding(max_features, 64, input_length=maxlen))
model.add(Conv1D(64, 7, activation='relu'))
# model.add(MaxPooling1D(5))
# model.add(Conv1D(64, 7, activation='relu'))
# model.add(LSTM(64,
#                dropout=0.5,
#                recurrent_dropout=0.5,
#                return_sequences=True))
model.add(LSTM(128,
               dropout=0.5,
               recurrent_dropout=0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
# model.add(Dense(128, activation='relu'))
model.add(Dense(num_categories, activation='softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_51 (Embedding)     (None, 100, 64)           574848    
_________________________________________________________________
conv1d_46 (Conv1D)           (None, 94, 64)            28736     
_________________________________________________________________
lstm_33 (LSTM)               (None, 128)               98816     
_________________________________________________________________
dense_78 (Dense)             (None, 128)               16512     
_________________________________________________________________
dropout_26 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_79 (Dense)             (None, 46)                5934      
Total params: 724,846
Trainable params: 724,846
Non-trainable params: 0
_________________________________________________________________


In [138]:
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['acc'])

history = model.fit(x_train, y_train,
                    epochs=10,
                    batch_size=32,
                    validation_split=0.2)

Train on 7185 samples, validate on 1797 samples
Epoch 1/10
7185/7185 [==============================] - 98s 14ms/step - loss: 2.1801 - acc: 0.4305 - val_loss: 1.8254 - val_acc: 0.5125
Epoch 2/10
7185/7185 [==============================] - 75s 10ms/step - loss: 1.8556 - acc: 0.5083 - val_loss: 1.7905 - val_acc: 0.5320
Epoch 3/10
7185/7185 [==============================] - 75s 10ms/step - loss: 1.7673 - acc: 0.5326 - val_loss: 1.7945 - val_acc: 0.5398
Epoch 4/10
7185/7185 [==============================] - 74s 10ms/step - loss: 1.6691 - acc: 0.5580 - val_loss: 1.6666 - val_acc: 0.5726
Epoch 5/10
7185/7185 [==============================] - 73s 10ms/step - loss: 1.5710 - acc: 0.5818 - val_loss: 1.7452 - val_acc: 0.5715
Epoch 6/10
7185/7185 [==============================] - 75s 10ms/step - loss: 1.4977 - acc: 0.6060 - val_loss: 1.6666 - val_acc: 0.5860
Epoch 7/10
7185/7185 [==============================] - 74s 10ms/step - loss: 1.4433 - acc: 0.6157 - val_loss: 1.6414 - val_acc: 0.5977


In [139]:
model.save_weights('combo_model_v2.h5')

# Testing Final Model

In [145]:
model = Sequential()
model.add(Embedding(max_features, 64, input_length=maxlen))
model.add(Conv1D(64, 7, activation='relu'))
model.add(MaxPooling1D(5))
model.add(Conv1D(64, 7, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dropout(0.5))
model.add(Dense(num_categories, activation='softmax'))

model.load_weights('1d_cnn_model.h5')

model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['acc'])

model.evaluate(x_test, y_test)

2246/2246 [==============================] - 8s 4ms/step


[1.8121196348966495, 0.6923419412819276]